In [ ]:
import pandas as pd
from pumping_cycle_utils import append_dataframe, populate_tresholds, populate_statistics

df = pd.read_csv("Pressure.csv")
df["Date"] = pd.to_datetime(df["Date"])

print("{0}Pumping Cycle Tresholds Results{0}".format("="*10))
#Add 2 new columns to original DF
append_dataframe(df,start_time_offset=10,end_time_offset=-5)
populate_tresholds(dataframe=df)

print("{0}Pumping Cycle Health Analysis Results{0}".format("="*10))
#Show Kendalls Tau and p-value for all pump cycles
populate_statistics(dataframe=df)


